In [2]:
%matplotlib inline
# Import required libraries

import os
import time
import sys

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image
from torch.utils.data import random_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import urllib
import numpy as np
import matplotlib.pyplot as plt
import glob

In [3]:
# This is a tool I have provided you to help you download your file.

def download_file(url, filename):
    """
    A function that downloads the data file from a URL
    Parameters
    ----------
    url : string
        url where the file to download is located
    filename : string
        location where to save the file
    reporthook : function
        callback to display the download progress
    """
    if not os.path.isfile(filename):
        urllib.request.urlretrieve(url, filename, reporthook)
        
def reporthook(count, block_size, total_size):
    """
    A function that displays the status and speed of the download
    """

    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = int(progress_size / (1024 * duration + 0.0001))
    percent = int(count * block_size * 100 / total_size)
    sys.stdout.write("\r...%d%%, %d MB, %d KB/s, %d seconds passed" %
                     (percent, progress_size / (1024 * 1024), speed, duration))
    sys.stdout.flush()

In [4]:
# You can download your file by typing your first name into the name block
# The name used is the first part of your first name as listed in BB learn
# If you have problems downloading the data please reach out to me

name = 'Dhruv'
# download_file(f'https://zenodo.org/record/7339649/files/data_{name}.npz?download=1','data.npz')

In [5]:
# Load the data

data = np.load('data.npz')
data_list = data.files
for item in data_list:
    print(f"{item}: {data[item].shape}")

# print(data["training_true"])

training_feat: (100000, 30)
training_true: (100000, 3)
validation_feat: (65536, 30)


In [14]:
# Create tensors from numpy arrays
training_feat_np = data[data_list[0]]
scaler = StandardScaler()
training_feat_scaled = scaler.fit(training_feat_np)
training_feat_scaled = scaler.transform(training_feat_np)
X = torch.from_numpy(training_feat_scaled)

training_true_np = data[data_list[1]]
y = torch.from_numpy(training_true_np)

validation_feat_np = data[data_list[2]]
X_test = torch.from_numpy(validation_feat_np)

tensor(-8.0945e-17, dtype=torch.float64)


In [ ]:
class Data(Dataset):
  '''Dataset Class to store the samples and their corresponding labels, 
  and DataLoader wraps an iterable around the Dataset to enable easy access to the samples.
  '''

  def __init__(self, X: np.ndarray, y: np.ndarray, device = 'cpu') -> None:

    # need to convert float64 to float32 else 
    # will get the following error
    # RuntimeError: expected scalar type Double but found Float
    self.X = torch.from_numpy(X.astype(np.float32)).to(device)
    self.y = torch.from_numpy(y.astype(np.float32)).to(device)
    self.len = self.X.shape[0]
  
  def __getitem__(self, index: int) -> tuple:
    return self.X[index], self.y[index]

  def __len__(self) -> int:
    return self.len

In [ ]:
# Split dataset into test/train
split_dataset = random_split(training_feat_np, 
                             lengths=[2/3, 1/3], 
                             generator=torch.Generator().manual_seed(42))

# Configure dataset
training_data = Data(X=training_feat_np, y=training_true_np)
testing_data = Data(X=validation_feat_np, y=training_true_np)

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
# test_dataloader = DataLoader(testing_data, batch_size=batch_size, shuffle=True)

for X, y in train_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 30])
Shape of y: torch.Size([64, 3]) torch.float32


In [ ]:
class Neural_Network(nn.Module):
  ''' Regression Model
  ''' 

  # note, you can ignore the `:int` and `-> None` this is just more advanced doctring syntax
  def __init__(self, input_dim: int, hidden_dim: int, output_dim: int) -> None:
      '''The network has 4 layers
            - input layer
            - ReLu
            - hidden layer
            - ReLu
            - hidden layer
            - ReLu
            - output layer
      '''
      super(Neural_Network, self).__init__()
      # in this part you should intantiate each of the layer components
      # Type your code here

  def forward(self, x: torch.Tensor) -> torch.Tensor:
      # In this part you should build a model that returns the 3 outputs of the regression
      # Type your code here
      
      return x